# Dos & Ddos Attack Detection (LSTM Model)

In [1]:
# importing the Required Moudules

import os 
import math
import pandas as pd
import numpy as  np
import tensorflow as tf

from tensorflow import keras
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, LabelEncoder # type: ignore
from tensorflow.keras.models import Sequential, save_model, load_model, Model # type: ignore
from tensorflow.keras.layers import InputLayer, LSTM, Flatten, Concatenate, Dense, Input # type: ignore
from tensorflow.keras.losses import MeanSquaredError # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # type: ignore

# Data Preparation

In [11]:
# Loading the Data Sets
main_dataset = pd.read_csv('C:/Users/srika/OneDrive/Desktop/DeepNet Guardian FrameWork/src/Dos Attacks/data/final_dataset.csv')

In [17]:
main_dataset.head(10)

,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,624,192.168.4.118-203.73.24.75-4504-80-6,192.168.4.118,4504,203.73.24.75,80,6,12/06/2010 08:34:32 AM,3974862,29,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
1,625,192.168.4.118-203.73.24.75-4504-80-6,192.168.4.118,4504,203.73.24.75,80,6,12/06/2010 08:34:36 AM,63,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
2,626,192.168.4.118-203.73.24.75-4505-80-6,192.168.4.118,4505,203.73.24.75,80,6,12/06/2010 08:34:36 AM,476078,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
3,627,192.168.4.118-203.73.24.75-4505-80-6,192.168.4.118,4505,203.73.24.75,80,6,12/06/2010 08:34:37 AM,151,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
4,628,192.168.4.118-203.73.24.75-4506-80-6,192.168.4.118,4506,203.73.24.75,80,6,12/06/2010 08:34:37 AM,472507,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
5,629,192.168.4.118-203.73.24.75-4506-80-6,192.168.4.118,4506,203.73.24.75,80,6,12/06/2010 08:34:37 AM,138,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
6,630,192.168.4.118-203.73.24.75-4507-80-6,192.168.4.118,4507,203.73.24.75,80,6,12/06/2010 08:34:37 AM,469596,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
7,631,192.168.4.118-203.73.24.75-4507-80-6,192.168.4.118,4507,203.73.24.75,80,6,12/06/2010 08:34:37 AM,1978,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
8,632,192.168.4.118-203.73.24.75-4508-80-6,192.168.4.118,4508,203.73.24.75,80,6,12/06/2010 08:34:37 AM,468629,2,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos
9,633,192.168.4.118-203.73.24.75-4508-80-6,192.168.4.118,4508,203.73.24.75,80,6,12/06/2010 08:34:38 AM,102,1,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ddos


In [33]:
# Select only the required features
selected_features = [
    "Timestamp",  # Time of the flow
    "Flow Pkts/s",  # Packet Rate
    "Pkt Size Avg",  # Packet Size
    "Flow Duration",  # Flow Duration
    "SYN Flag Cnt",  # TCP SYN Count
    "ACK Flag Cnt",  # TCP ACK Count
    "Down/Up Ratio",  # Ratio of incoming and outgoing traffic
    "Protocol",  # Needed to derive UDP & ICMP counts
    "Label"  # Attack or Normal
]

dataset = main_dataset[selected_features]

In [35]:
dataset.head(20)  

,Timestamp,Flow Pkts/s,Pkt Size Avg,Flow Duration,SYN Flag Cnt,ACK Flag Cnt,Down/Up Ratio,Protocol,Label
0,12/06/2010 08:34:32 AM,18.365417,820.506849,3974862,1,0,1.0,6,ddos
1,12/06/2010 08:34:36 AM,31746.031746,0.000000,63,0,1,1.0,6,ddos
2,12/06/2010 08:34:36 AM,16.803969,390.375000,476078,1,0,3.0,6,ddos
3,12/06/2010 08:34:37 AM,19867.549669,0.000000,151,0,1,0.0,6,ddos
4,12/06/2010 08:34:37 AM,14.814595,160.428571,472507,1,0,2.0,6,ddos
5,12/06/2010 08:34:37 AM,14492.753623,0.000000,138,0,1,1.0,6,ddos
6,12/06/2010 08:34:37 AM,14.906430,137.000000,469596,1,0,2.0,6,ddos
7,12/06/2010 08:34:37 AM,1011.122346,0.000000,1978,0,1,1.0,6,ddos
8,12/06/2010 08:34:37 AM,14.937189,105.571429,468629,1,0,2.0,6,ddos
9,12/06/2010 08:34:38 AM,19607.843137,0.000000,102,0,1,1.0,6,ddos


# Cleaning the Data Set

In [28]:
# # Converting the time stamp to seconds
# dataset["Timestamp"] = pd.to_datetime(dataset["Timestamp"])
# dataset["Timestamp"] = dataset["Timestamp"].dt.second

In [ ]:
import lable

In [ ]:
# Checking Null Values
null_values = dataset.isnull().sum()
print("Null values in each column:\n", null_values)

# Drop the Duplicate Values (cant check the duplicate values in the dataset since the pakage sizes are same in some cases)

# primt the each Cloumn Count Values 
dataset.count()

# laeling the label data to ddos to 1 and normal to 0
label_encoder = LabelEncoder()
dataset["Label"] = label_encoder.fit_transform(df["Label"])


# Data Normalization

In [ ]:
# Normalize the Data
scaler = MinMaxScaler()
feature_cols = df.columns[:-1]  # All columns except 'Label'
dataset[feature_cols] = scaler.fit_transform(dataset[feature_cols])

# Spliting the Data to the train and test

In [ ]:
X = dataset.drop(columns=["Label"])
# Convert labels to binary (DDoS = 1, Normal = 0)
y = dataset["Label"].apply(lambda x: 1 if x == "DDoS" else 0)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to 3D shape: (samples, timesteps=1, features)
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout

# Define the Bi-LSTM model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(1, X_train.shape[2])),
    Bidirectional(LSTM(32)),
    Dense(16, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # Binary classification (attack or normal)
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
